# SAR Wet Snow Analysis Using  Pyton GEE API

This notebook details the acquisition of Sentinel-1 SAR images from Google Earth Engine (GEE) and the subsequent wet-snow threshold analysis for Storm Darcy, in North East England 2021.

In [3]:
import ee
import geojson
import numpy as np
import re
import IPython.display as disp
import refined_lee_filter
from matplotlib import pyplot as plt
%matplotlib inline

# Shape file with north-east boundaries - will be used to get images of interest
ne_boundary_fname = 'north_east_4326.geojson'

with open(ne_boundary_fname) as f:
    ne_boundary = geojson.load(f)


In [4]:
# ee.Authenticate() # Do this just once
ee.Initialize()

In [5]:
# Get location details
# North east study area
coords = ne_boundary['features'][0]['geometry']['coordinates']
aoi = ee.Geometry.Polygon(coords)
newcastle_point = ee.Geometry.Point(-1.7969253, 55.0022917)

## Select date range
storm_darcy = ('2021-01-01', '2021-02-28')
beast_of_east = ('2018-02-24', '2018-03-04')

winter_spring_2021 = ('2020-12-01', '2021-02-28')

dates = winter_spring_2021

# Import the USGS ground elevation image.
elv = ee.Image('USGS/SRTMGL1_003')

def get_s1_collection(aoi, start_date, end_date, orbit_pass='ASCENDING', rel_orbit=30):
    """
    Get sentinel-1 image collection from GEE
    """
    return (ee.ImageCollection('COPERNICUS/S1_GRD_FLOAT') 
            .filterBounds(aoi)
            .filterDate(ee.Date(start_date), ee.Date(end_date)) 
            .filter(ee.Filter.eq('orbitProperties_pass', orbit_pass))
            .filter(ee.Filter.eq('relativeOrbitNumber_start', rel_orbit))
            .sort('system:time_start'))

# Sentinel-1 float - Storm Darcy
s1_coll = get_s1_collection(newcastle_point, dates[0], dates[1])

# MODIS snow cover 500m
modis_coll = (ee.ImageCollection('MODIS/006/MOD10A1')
            .filterBounds(newcastle_point)
            .filterDate(re.sub('^\d{4}', '2019', dates[0]), ee.Date(dates[1])) 
            .sort('system:time_start'))

# Landcover classifications
lc_2019 = (ee.Image('COPERNICUS/Landcover/100m/Proba-V-C3/Global/2019')
           .select('discrete_classification')
           .clip(aoi)
          )

In [6]:
import time

# Sen1 dates
acq_times_sar = s1_coll.aggregate_array('system:time_start').getInfo()
s1_dates = [time.strftime('%Y-%m-%d', time.gmtime(acq_time/1000)) for acq_time in acq_times_sar]

print(len(s1_dates))
print(s1_dates)

15
['2020-12-04', '2020-12-10', '2020-12-16', '2020-12-22', '2020-12-28', '2021-01-03', '2021-01-09', '2021-01-15', '2021-01-21', '2021-01-27', '2021-02-02', '2021-02-08', '2021-02-14', '2021-02-20', '2021-02-26']


In [7]:
def get_date_index(dte_list, dte):
    return [i for i in range(len(dte_list)) if dte_list[i] == dte][0]

def clip_img(img):
    """Clips a list of images."""
    return ee.Image(img).clip(aoi)

In [8]:
# Import the Folium library.
import folium

# Define a method for displaying Earth Engine image tiles to a folium map.
def add_ee_layer(self, ee_image_object, vis_params, name, show=True, opacity=1, min_zoom=0):
    map_id_dict = ee.Image(ee_image_object).getMapId(vis_params)
    folium.raster_layers.TileLayer(
        tiles=map_id_dict['tile_fetcher'].url_format,
        attr='Map Data &copy; <a href="https://earthengine.google.com/">Google Earth Engine</a>',
        name=name,
        show=show,
        opacity=opacity,
        min_zoom=min_zoom,
        overlay=True,
        control=True
        ).add_to(self)

# Add EE drawing method to folium.
folium.Map.add_ee_layer = add_ee_layer

In [10]:
# Plot the land classifications
location = aoi.centroid().coordinates().getInfo()[::-1]

# Create the map object.
m = folium.Map(location=location, zoom_start=12)

# Add the S1 rgb composite to the map object.
m.add_ee_layer(lc_2019, {}, 'Land Cover')

# # Add a layer control panel to the map.
m.add_child(folium.LayerControl())

# Display the map.
display(m)

In [13]:
# Create masks - water mask and relative orbit 30 mask to 
# SAR dates
im_list_s1 = s1_coll.toList(s1_coll.size()).map(clip_img)
im2_s1 = ee.Image(im_list_s1.get(get_date_index(s1_dates, '2021-02-08'))).select(['VV', 'VH']).clip(aoi) # snow

# Water mask from landcover classifications
water_mask = lc_2019.select('discrete_classification').eq([200, 80])
# Orbit 30 mask
rel_orbit_mask = im2_s1.select('VV').gte(0)

## Sentinel-1 SAR data

In [23]:
SNOW_DIFF_TRESH_DB = -2
AOI = aoi

def get_s1_collection(aoi, start_date, end_date, orbit_pass='ASCENDING', rel_orbit=30):
    """
    Get sentinel-1 image collection from GEE
    """
    return (ee.ImageCollection('COPERNICUS/S1_GRD_FLOAT') 
            .filterBounds(aoi)
            .filterDate(ee.Date(start_date), ee.Date(end_date)) 
            .filter(ee.Filter.eq('orbitProperties_pass', orbit_pass))
            .filter(ee.Filter.eq('relativeOrbitNumber_start', rel_orbit))
            .sort('system:time_start'))

def add_diff_bands(img1, img2):
    """
    Add differences of VV and VH polarities between img2 (snow) and img1 (no snow) to img2
    """
    # Get S1 VV difference between image 1 and image 2
    img2_vv_diff = (img2.select('VV')
                    .subtract(img1.select('VV'))
                    .reproject(**{'crs': img2.select([0]).projection(), 'scale': 20})
                    .rename('wet_snow_vv'))
    # Get S1 VH bands from image 1 and image 2
    img2_vh_diff = (img2.select('VH')
                    .subtract(img1.select('VH'))
                    .reproject(**{'crs': img2.select([0]).projection(), 'scale': 20})
                    .rename('wet_snow_vh'))
    # Get S1 VH bands from image 1 and image 2
    img2_vhvv_diff = (img2.select('VH')
                    .subtract(img1.select('VV'))
                    .reproject(**{'crs': img2.select([0]).projection(), 'scale': 20})
                    .rename('wet_snow_vhvv'))

    # Condition decibel difference between images less than threshold
    is_wet_snow_vv = img2_vv_diff.lt(SNOW_DIFF_TRESH_DB).rename('wet_snow_vv_mask')
    is_wet_snow_vh = img2_vh_diff.lt(SNOW_DIFF_TRESH_DB).rename('wet_snow_vh_mask')
    is_wet_snow_vhvv = img2_vhvv_diff.lt(SNOW_DIFF_TRESH_DB).rename('wet_snow_vhvv_mask')
    
    # Add VV, VH differences and 
    return img2.addBands(ee.Image([img2_vv_diff, img2_vh_diff, img2_vhvv_diff, is_wet_snow_vv, is_wet_snow_vh, is_wet_snow_vhvv]))

def display_wet_snow_layers(img, aoi=None):
    
    # Subset layers and prepare them for display.
    wet_snow_vv_mask = img.select('wet_snow_vv_mask').selfMask()
    wet_snow_vh_mask = img.select('wet_snow_vh_mask').selfMask()
    wet_snow_vhvv_mask = img.select('wet_snow_vhvv_mask').selfMask()
    wet_snow_vv = img.select('wet_snow_vv')
    wet_snow_vh = img.select('wet_snow_vh')
    wet_snow_vhvv = img.select('wet_snow_vhvv')

    # Create a folium map object.
    if aoi is not None:
        coords = aoi['features'][0]['geometry']['coordinates']
        center = (ee.Geometry.Polygon(coords)
                  .centroid(10)
                  .coordinates()
                  .reverse()
                  .getInfo())
    else:
        center = AOI.centroid(10).coordinates().reverse().getInfo()
    
    m = folium.Map(location=center, zoom_start=12)

    # Add layers to the folium map.
    # Add the S1 rgb composite to the map object.
#     m.add_ee_layer(rgb, {'min': [-20, -20, 0], 'max': [0, 0, 1]}, 'FFA')
    
    m.add_ee_layer(wet_snow_vv,
                   {'min': -20, 'max': 5, 'palette': ['white', 'black']},
                   'wet_snow_vv', False, 1, 9)
    
    m.add_ee_layer(wet_snow_vh,
                   {'min': -20, 'max': 5, 'palette': ['white', 'black']},
                   'wet_snow_vh', False, 1, 9)
    
    m.add_ee_layer(wet_snow_vhvv,
                   {'min': -20, 'max': 5, 'palette': ['white', 'black']},
                   'wet_snow_vhvv', False, 1, 9)
    
    m.add_ee_layer(wet_snow_vv_mask, {'palette': 'orange'},
                   'wet_snow_mask_vv', True, 1, 9)
    
    m.add_ee_layer(wet_snow_vh_mask, {'palette': 'red'},
                   'wet_snow_mask_vh', True, 1, 9)
    
    m.add_ee_layer(wet_snow_vhvv_mask, {'palette': 'purple'},
                   'wet_snow_mask_vhvv', False, 1, 9)
    
    
    # Add AOI polygon layer
    if aoi is not None:
        # Add north east boundary to map
        ne_poly = folium.GeoJson(data=aoi,
                                 style_function=lambda x: {'lineColor': 'black'},
                                 name='Durham AOI')
        ne_poly.add_to(m)

    # Add a layer control panel to the map.
    m.add_child(folium.LayerControl())

    # Display the map.
    display(m)


## MODIS Snow Cover

### Storm Darcy

In [24]:
modis_list = modis_coll.toList(modis_coll.size()).map(clip_img)
modis_dates = [time.strftime('%Y-%m-%d', time.gmtime(acq_time/1000))
               for acq_time in modis_coll.aggregate_array('system:time_start').getInfo()]
modis_indexes = [get_date_index(modis_dates, d) for d in s1_dates]

In [25]:
s1_dates

['2020-12-04',
 '2020-12-10',
 '2020-12-16',
 '2020-12-22',
 '2020-12-28',
 '2021-01-03',
 '2021-01-09',
 '2021-01-15',
 '2021-01-21',
 '2021-01-27',
 '2021-02-02',
 '2021-02-08',
 '2021-02-14',
 '2021-02-20',
 '2021-02-26']

In [26]:
# Total number of MODIS 500m pixels in image
total_pixel_count = ee.Image(modis_list.get(0)).reduceRegion(ee.Reducer.count(), aoi, scale = 500).get('NDSI').getInfo()
total_pixel_count

34138

In [19]:
# Find reference images for no snow days to average
no_snow_idx = []
for idx in modis_indexes:
    print(modis_dates[idx])
    img = ee.Image(modis_list.get(idx))
    # Get mean value of NDSI pixels and overall count
    # we want to keep images which have a low NDSI and high count
    mean = img.reduceRegion(ee.Reducer.mean(), aoi).get('NDSI_Snow_Cover').getInfo()
    pixel_count = img.reduceRegion(ee.Reducer.count(), aoi).get('NDSI_Snow_Cover').getInfo()
    
    if mean is None:
        print(f'Insufficient data available for {modis_dates[idx]}')
    elif mean < 10 and (pixel_count / total_pixel_count) > 0.2:
        no_snow_idx.append(idx)
        print(f'**Accepted**\nMean NDSI is: {mean}\nPct pixels is: {pixel_count / total_pixel_count}')
    else:
        print(f'**Rejected**\nMean NDSI is: {mean}\nPct pixels is: {pixel_count / total_pixel_count}')
    print('\n')

2020-12-04
Insufficient data available for 2020-12-04


2020-12-10
**Rejected**
Mean NDSI is: 50.699284713912384
Pct pixels is: 0.021413088054367566


2020-12-16
Insufficient data available for 2020-12-16


2020-12-22
**Accepted**
Mean NDSI is: 3.878870944791534
Pct pixels is: 0.44914757747964146


2020-12-28
**Rejected**
Mean NDSI is: 21.81452080522602
Pct pixels is: 0.28015700978381863


2021-01-03
**Rejected**
Mean NDSI is: 42.59896406013346
Pct pixels is: 0.22057531196906674


2021-01-09
Insufficient data available for 2021-01-09


2021-01-15
**Rejected**
Mean NDSI is: 44.88211370695625
Pct pixels is: 0.5074696818794305


2021-01-21
**Rejected**
Mean NDSI is: 18.09106354498445
Pct pixels is: 0.24069951373835607


2021-01-27
**Accepted**
Mean NDSI is: 4.55771298487253
Pct pixels is: 0.24046517077743276


2021-02-02
**Rejected**
Mean NDSI is: 49.38916609517506
Pct pixels is: 0.014558556447360713


2021-02-08
**Rejected**
Mean NDSI is: 53.33353102725139
Pct pixels is: 0.27204288476184

In [27]:
# Adapted from Kuik's answer https://gis.stackexchange.com/questions/363857/using-date-list-to-filter-image-collection-in-google-earth-engine

def return_ee_date(dte):
    return ee.Date(dte).millis()

def return_simpletime_date(img):
    return img.set('simpleTime', ee.Date(img.date().format('YYYY-MM-dd')).millis())

no_snow_dates = ee.List([modis_dates[idx] for idx in no_snow_idx]).map(return_ee_date)

s1_coll_no_snow = s1_coll.map(return_simpletime_date)
s1_coll_no_snow = s1_coll_no_snow.filter(ee.Filter.inList("simpleTime", no_snow_dates))


In [28]:
# Check dates of no snow reference images
[modis_dates[i] for i in no_snow_idx]

['2020-12-22', '2021-01-27', '2021-02-26']

In [29]:
# Projections of original sentinel1 data and MODIS
s1_projection = ee.Image(im_list_s1.get(get_date_index(s1_dates, '2021-02-08'))).select('VV').projection()
modis_projection = ee.Image(modis_list.get(get_date_index(modis_dates, '2021-02-08'))).projection()

# Compute average of snow-free images
s1_no_snow_avg = s1_coll_no_snow.mean().setDefaultProjection(s1_projection)
s1_no_snow_avg_db = s1_no_snow_avg.log10().multiply(10)


comparison_date = '2021-02-08'

# Convert to decibels and find areas where difference is > 2dB
s1_snow_db = (ee.Image(im_list_s1.get(get_date_index(s1_dates, comparison_date)))
              .select(['VV', 'VH'])
              .clip(aoi)
              .log10()
              .multiply(10))

s1_no_snow_db_rs = (s1_no_snow_avg_db
                    .reproject(
        crs = modis_projection
    )
    # Force the next reprojection to aggregate instead of resampling.
    .reduceResolution(
      reducer = ee.Reducer.mean(),
      maxPixels = 1024)
    # Request the data at the scale and projection of the MODIS image.
    )

s1_snow_db_rs = (s1_snow_db
    # Force the next reprojection to aggregate instead of resampling.
    # Request the data at the scale and projection of the MODIS image.
    .reproject(
        crs = modis_projection
    )
                .reduceResolution(
      reducer = ee.Reducer.mean(),
      maxPixels = 1024))

# diff_s1_db = add_diff_bands(s1_coll_no_snow_avg_db, s1_snow_db)
diff_s1_db = add_diff_bands(s1_no_snow_db_rs, s1_snow_db_rs)

# Mask water features
diff_s1_db_msk = diff_s1_db.updateMask(water_mask.select('constant_1').eq(0))
# Mask rel orbit
diff_s1_db_msk = diff_s1_db.updateMask(rel_orbit_mask.select('VV'))

display_wet_snow_layers(diff_s1_db_msk, ne_boundary)

In [30]:
# Get counts of pixels which are classified as 'wet snow' from the threshold analysis for VV and VH

vv_extent_cnt = diff_s1_db_msk.select('wet_snow_vv_mask').selfMask().reduceRegion(
        ee.Reducer.count(),
        aoi,
        scale = 500,
        maxPixels = 1e13
    ).getInfo()

vh_extent_cnt = diff_s1_db_msk.select('wet_snow_vh_mask').selfMask().reduceRegion(
        ee.Reducer.count(),
        aoi,
        scale = 500,
        maxPixels = 1e13
    ).getInfo()

total_extent_cnt = diff_s1_db_msk.select('VV').reduceRegion(
        ee.Reducer.count(),
        aoi,
        scale = 500,
        maxPixels = 1e13
    ).getInfo()

print(list(vv_extent_cnt.values())[0], list(vv_extent_cnt.values())[0]/list(total_extent_cnt.values())[0])
print(list(vh_extent_cnt.values())[0], list(vh_extent_cnt.values())[0]/list(total_extent_cnt.values())[0])
print(total_extent_cnt)

8878 0.2876863253402463
16330 0.5291639662994168
{'VV': 30860}


In [35]:
# Plot 
# Define a palette for the 18 distinct land cover classes.
ndsi_class_pal = [
    'red', # Missing data
    'green', # No decision
    'purple', # Night
    'yellow', # Inland water
    'blue', # Ocean
    'orange', # Cloud
    'black' # Detector saturated
]

# Plot Sen1 wet snow and MODIS
img = diff_s1_db_msk

# Subset layers and prepare them for display.
wet_snow_vv_mask = img.select('wet_snow_vv_mask').selfMask()
wet_snow_vh_mask = img.select('wet_snow_vh_mask').selfMask()
wet_snow_vv = img.select('wet_snow_vv')
wet_snow_vh = img.select('wet_snow_vh')

# Create a folium map object.
center = aoi.centroid(10).coordinates().reverse().getInfo()
m = folium.Map(location=center, zoom_start=12)

# Add layers to the folium map.
# Add AOI polygon layer
# Add north east boundary to map
ne_poly = folium.GeoJson(data=ne_boundary,
                         style_function=lambda x: {'lineColor': 'black'},
                         name='North East AOI')
ne_poly.add_to(m)

# Sentinel-1
m.add_ee_layer(wet_snow_vv,
               {'min': -20, 'max': 5, 'palette': ['white', 'black']},
               'wet_snow_vv', False, 1, 9)

m.add_ee_layer(wet_snow_vh,
               {'min': -20, 'max': 5, 'palette': ['white', 'black']},
               'wet_snow_vh', False, 1, 9)

m.add_ee_layer(wet_snow_vv_mask, {'palette': 'orange'},
               'wet_snow_mask_vv', True, 1, 9)

m.add_ee_layer(wet_snow_vh_mask, {'palette': 'red'},
               'wet_snow_mask_vh', True, 1, 9)

# MODIS
img_modis = ee.Image(modis_list.get(get_date_index(modis_dates, '2021-02-08')))

# Mask for rel orbit
img_modis = img_modis.updateMask(rel_orbit_mask)

# Add layers to the folium map.
m.add_ee_layer(img_modis,
               {'bands': ['NDSI_Snow_Cover'], 'min':0, 'max':100},
               'Modis Snow', True, 1, 9)

m.add_ee_layer(img_modis,
               {'bands': ['NDSI_Snow_Cover_Class'], 'min':200, 'max':254, 'palette':ndsi_class_pal},
               'Modis Snow Class', True, 1, 9)

# Add a layer control panel to the map.
m.add_child(folium.LayerControl())

# Display the map.
display(m)

In [38]:
# Add the NDSI snow cover to the bands to count pixels
diff_s1_db_msk = diff_s1_db_msk.addBands(img_modis.select('NDSI_Snow_Cover'))

# Mask cloud pixels and add bands
modis_snow_mask = img_modis.select('NDSI_Snow_Cover').gt(0).rename('modis_snow_mask')
diff_s1_db_msk_cld = diff_s1_db_msk.addBands(modis_snow_mask)
diff_s1_db_msk_cld = diff_s1_db_msk_cld.updateMask(modis_snow_mask)

# Plot data
img = diff_s1_db_msk_cld

# Create a folium map object.
center = aoi.centroid(10).coordinates().reverse().getInfo()
m = folium.Map(location=center, zoom_start=12)

# Add layers to the folium map.
m.add_ee_layer(img.select('wet_snow_vv_mask').selfMask(),
               {
#                    'bands': ['wet_snow_vv_mask'],
                   'palette': 'red'},
               'Wet Snow VV', True, 1, 9)

# Add layers to the folium map.
m.add_ee_layer(img.select('modis_snow_mask').selfMask(),
               {
#                    'bands': ['wet_snow_vv_mask'],
                   'palette': 'blue'},
               'MODIS Snow Mask', True, 1, 9)

# Add layers to the folium map.
m.add_ee_layer(diff_s1_db_msk.select('wet_snow_vv_mask').selfMask(),
               {
#                    'bands': ['wet_snow_vv_mask'],
                   'palette': 'purple'},
               'Wet Snow VV orig', True, 1, 9)

# Add a layer control panel to the map.
m.add_child(folium.LayerControl())

# Display the map.
display(m)
# m.save("filename.html")

In [39]:
# Add original vv/vh masks to output raster
wet_snow_vv_mask_orig = diff_s1_db_msk.select('wet_snow_vv_mask').rename('wet_snow_vv_mask_orig')
wet_snow_vh_mask_orig = diff_s1_db_msk.select('wet_snow_vh_mask').rename('wet_snow_vh_mask_orig')

diff_s1_db_msk_cld = diff_s1_db_msk_cld.addBands([wet_snow_vv_mask_orig, wet_snow_vh_mask_orig])

# Add cloud cover mask
cloud_cover = diff_s1_db_msk_cld.select('modis_snow_mask').eq(0).rename('cloud_cover')
diff_s1_db_msk_cld = diff_s1_db_msk_cld.addBands(cloud_cover)

In [40]:
# Check bands have been added
diff_s1_db_msk_cld.bandNames().getInfo()

['VV',
 'VH',
 'wet_snow_vv',
 'wet_snow_vh',
 'wet_snow_vhvv',
 'wet_snow_vv_mask',
 'wet_snow_vh_mask',
 'wet_snow_vhvv_mask',
 'NDSI_Snow_Cover',
 'NDSI_Snow_Cover_1',
 'modis_snow_mask',
 'wet_snow_vv_mask_orig',
 'wet_snow_vh_mask_orig',
 'cloud_cover']

In [41]:
# Get pixels which are classified as both snow from sentinel and snow from MODIS
s1_vv_snow = diff_s1_db_msk_cld.select('wet_snow_vv_mask').eq(1).selfMask() #.and(diff_s1_db_msk.select('NDSI_Snow_Cover').gte(1)))
s1_vv_nosnow = diff_s1_db_msk_cld.select('wet_snow_vv_mask').eq(0).selfMask()
s1_vh_snow = diff_s1_db_msk_cld.select('wet_snow_vh_mask').eq(1).selfMask() #.and(diff_s1_db_msk.select('NDSI_Snow_Cover').gte(1)))
s1_vh_nosnow = diff_s1_db_msk_cld.select('wet_snow_vh_mask').eq(0).selfMask()
modis_snow = diff_s1_db_msk_cld.select('modis_snow_mask').eq(1).selfMask() 


In [44]:
# Total count of Sentinel-1 snow and no-snow pixels for VV and VH

def get_pixel_count(gee_object, aoi, scale=500, maxpixels=1e13):
    """Function wrapper for ee.reduceRegion count"""
    return gee_object.reduceRegion(
        ee.Reducer.count(),
        aoi,
        scale = scale,
        maxPixels = maxpixels
    ).getInfo()

# VV snow / no snow
s1_vv_snow_cnt = get_pixel_count(s1_vv_snow, aoi)
s1_vv_no_snow_cnt = get_pixel_count(s1_vv_nosnow, aoi)

# VH snow / no snow
s1_vh_snow_cnt = get_pixel_count(s1_vh_snow, aoi)
s1_vh_no_snow_cnt = get_pixel_count(s1_vh_nosnow, aoi)

# Total MODIS snow pixels
modis_snow_cnt = get_pixel_count(modis_snow, aoi)

print(f"S1 VV -2dB snow pixels: {list(s1_vv_snow_cnt.values())[0]}, {list(s1_vv_snow_cnt.values())[0]/list(modis_snow_cnt.values())[0]}")
print(f"S1 VV -2dB no snow pixels: {list(s1_vv_no_snow_cnt.values())[0]}, {list(s1_vv_no_snow_cnt.values())[0]/list(modis_snow_cnt.values())[0]}")
print(f"S1 VH -2dB snow pixels: {list(s1_vh_snow_cnt.values())[0]}, {list(s1_vh_snow_cnt.values())[0]/list(modis_snow_cnt.values())[0]}")
print(f"S1 VH -2dB no snow pixels: {list(s1_vh_no_snow_cnt.values())[0]}, {list(s1_vh_no_snow_cnt.values())[0]/list(modis_snow_cnt.values())[0]}")
print(f"MODIS snow pixels: {list(modis_snow_cnt.values())[0]}, {list(modis_snow_cnt.values())[0]/list(modis_snow_cnt.values())[0]}")

S1 VV -2dB snow pixels: 2403, 0.3244228432563791
S1 VV -2dB no snow pixels: 4998, 0.6747671121911705
S1 VH -2dB snow pixels: 4190, 0.5656811124611854
S1 VH -2dB no snow pixels: 3211, 0.4335088429863643
MODIS snow pixels: 7407, 1.0


In [104]:
# Add bands to output file
diff_s1_db_msk_cld = diff_s1_db_msk_cld.addBands([s1_vv_snow.rename('s1_vv_snow'), s1_vv_nosnow.rename('s1_vv_nosnow'), modis_snow.rename('modis_snow')])

In [106]:
# Download image to disk
im = diff_s1_db_msk_cld
task = ee.batch.Export.image.toDrive(image=im.toFloat(),
                                     description=f'20210208_darcy',
                                     scale=500,
                                     region=aoi,
                                     folder='s1',
                                     fileNamePrefix='20210208_darcy_diff',
                                     crs='EPSG:4326',
                                     fileFormat='GeoTIFF')
task.start()